In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Water Saturation Estimation

Water saturation estimation is crucial in petrophysics for several reasons:

1. **Hydrocarbon Volume Calculation**: It helps determine the volume of hydrocarbons in place. Accurate water saturation (Sw) values are essential for calculating the original oil in place (OOIP) and original gas in place (OGIP) volumes¹(https://petroshine.com/fluid-saturation/).
2. **Reservoir Characterization**: Understanding the distribution of water saturation helps in characterizing the reservoir, which is vital for planning production strategies and enhancing recovery¹(https://petroshine.com/fluid-saturation/).
3. **Production Forecasting**: Sw values are used in reservoir models to predict future production and to evaluate the economic viability of the reservoir²(https://www.mdpi.com/2077-1312/9/6/666).

### Methods to Estimate Water Saturation

1. **Resistivity Logs**: This is the most common method, where water saturation is estimated using resistivity measurements from well logs. The Archie equation is often used for clean sands, while modified versions like the Waxman-Smits model are used for shaly sands³(https://petrowiki.spe.org/Water_saturation_determination).
2. **Capillary Pressure Measurements**: Laboratory measurements of capillary pressure and corresponding water saturation provide detailed information about the pore structure and fluid distribution³(https://petrowiki.spe.org/Water_saturation_determination).
3. **Core Analysis**: Direct measurement of water saturation from core samples using techniques like the Dean-Stark method³(https://petrowiki.spe.org/Water_saturation_determination).
4. **Nuclear Magnetic Resonance (NMR)**: NMR logging tools can provide estimates of water saturation by measuring the response of hydrogen nuclei in the formation fluids³(https://petrowiki.spe.org/Water_saturation_determination).

This notebook estimates the water saturation using Archie equation and saturation height function based on the capillary pressure measurement.


## Log Derived Water Saturation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

all_data = project.get_all_data()

focused_well = 'HW-29'
well_data = all_data[all_data.WELL_NAME == focused_well].copy()

In [ ]:
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation

# Debug water saturation
water_salinity = 2e5
m = 2

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

swt = archie_saturation(well_data['RT'], rw, well_data['PHIT'], m=m)

plt.figure(figsize=(10, 2))
plt.plot(well_data['DEPTH'], swt)
plt.ylim(0, 1)


## Saturation Height Function

### Core Data

Explain the data source,
- measurement techniques
- data qc is discussed in the next section

### Define Rock Type

Define the rock type based on FZI cut-offs from previous notebook



In [ ]:
from quick_pp.rock_type import calc_fzi, rock_typing, calc_r35
from quick_pp.core_calibration import fit_j_curve, j_xplot, leverett_j, sw_shf_leverett_j, poroperm_xplot, pc_xplot

core_data = pd.read_csv(r'data\01_raw\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['PC_RES'] = core_data['O/B Pc (psia)'] * 0.088894  # oil-brine system
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']
core_data['SWN'] = core_data.groupby('Sample')['SW'].transform(lambda x: (x - x.min()) / (1 - x.min()))

# Filter data
conditions = (
    (core_data['K mD'] > 0)
    & (core_data['Class'] == 'Good')
    # & (core_data['PC'] <= 40)
)
core_data = core_data[conditions].copy()

core_data.drop_duplicates(subset=['CPORE', 'CPERM', 'SW'], keep='last', inplace=True)

# Estimate rock types
r35 = calc_r35(core_data['CPORE'], core_data['CPERM'])
core_data['R35'] = r35
r35_cut_offs = [.1, .5, 2, 10, 100]
rock_flag = rock_typing(r35, higher_is_better=True, cut_offs=r35_cut_offs)
core_data['ROCK_FLAG'] = rock_flag

# Plot Pc curve for each sample
for sample, data in core_data.groupby('Sample'):
    pc_xplot(data['SW'], data['PC'], label=sample)

## Leverett J

Explain Leverett J technique

### Calculate J

In [ ]:
ift = 32
theta = 30

core_data['J'] = leverett_j(core_data['PC_RES'], ift, theta, core_data['CPERM'], core_data['CPORE'])
j_xplot(core_data['SW'], core_data['J'], core_group=core_data['ROCK_FLAG'])

In [ ]:
j_xplot(core_data['SWN'], core_data['J'], core_group=core_data['ROCK_FLAG'])

## QC the Pc data

The capillary pressure measurements for each Sample are plotted on a log-log plot.
The data points should fall on a relatively straight line indicating good data quality.

Based 
select the dataset for each rock type
curve fitting

In [ ]:
from ipywidgets import interact, widgets

rt = widgets.Dropdown(
    options=core_data['ROCK_FLAG'].unique(),
    value=1,
    description='Rock Type:'
)
a = widgets.FloatSlider(
    value=.01,
    min=.001,
    max=.1,
    step=.001,
    readout_format='.3f'
)
b = widgets.FloatSlider(
    value=.5,
    min=.1,
    max=3,
    step=.1,
    readout_format='.1f'
)

@interact(rt=rt, a=a, b=b)
def param(rt, a, b):
    data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(data['SWN'], data['J'], a=a, b=b, core_group=data['Sample'], log_log=False)

In [ ]:
params = {
    1: (.007, 1.2),
    2: (.009, 1.25),
    3: (.011, 1.26),
    4: (.015, 1.27),
}

for rt, param in params.items():
    a, b = param
    rock_data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(rock_data['SWN'], rock_data['J'], a=a, b=b, core_group=rock_data['Sample'],
            label=f'{rt}: a:{a}, b:{b}', log_log=False)
    plt.show()

## Estimate SHF

In [ ]:
params = {
    1: (.007, 1.2),
    2: (.009, 1.25),
    3: (.011, 1.26),
    4: (.015, 1.27),
}

ift = 32
theta = 30
ghc = .837
gw = 1.0
fwl = 8380

fwl = widgets.FloatSlider(
    value=fwl,
    min=fwl / 2,
    max=fwl * 2,
    step=5
)

@interact(fwl=fwl)
def plot(fwl):
    well_data['a'] = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[0])
    well_data['b'] = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[1])
    shf = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=well_data['a'], b=well_data['b'])

    plt.figure(figsize=(10, 2))
    plt.plot(well_data['DEPTH'], swt, label='SWT')
    plt.plot(well_data['DEPTH'], shf, label='SHF')
    plt.ylim(0, 1)
    plt.legend()

## Plot the results

In [ ]:
from quick_pp.plotter import plotly_log
fwl = 8380
shf = sw_shf_leverett_j(
    well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
    fwl=fwl, ift=ift, theta=theta, a=well_data['a'], b=well_data['b']
)
# Plot individual results
well_data['SWT'] = swt
well_data['SHF'] = shf
fig = plotly_log(well_data, '')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()